In [ ]:

import os
import shutil

import tf
from keras.src.layers import Resizing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np

# Đường dẫn thư mục gốc chứa dữ liệu
data_dir = r'C:\Users\ADMIN\PycharmProjects\pythonProject3\Data'

# Đường dẫn đến các thư mục train, validation, test
train_dir = r'C:\Users\ADMIN\PycharmProjects\pythonProject3\train'
validation_dir = r'C:\Users\ADMIN\PycharmProjects\pythonProject3\val'
test_dir = r'C:\Users\ADMIN\PycharmProjects\pythonProject3\test'

# 1. Chia dữ liệu thành train, validation và test
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        files = os.listdir(label_dir)
        train_files, remaining_files = train_test_split(files, test_size=0.3, random_state=42)
        validation_files, test_files = train_test_split(remaining_files, test_size=0.5, random_state=42)

        for file in train_files:
            src = os.path.join(label_dir, file)
            dst = os.path.join(train_dir, label, file)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy2(src, dst)

        for file in validation_files:
            src = os.path.join(label_dir, file)
            dst = os.path.join(validation_dir, label, file)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy2(src, dst)

        for file in test_files:
            src = os.path.join(label_dir, file)
            dst = os.path.join(test_dir, label, file)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy2(src, dst)

print("Dữ liệu đã được chia thành train, validation, test thành công.")

# 2. Tăng cường dữ liệu
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# 3. Sử dụng Transfer Learning (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# 4. Xây dựng mô hình
model = Sequential([
    base_model,
    Resizing(224, 224),
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# 5. Biên dịch mô hình
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6. Huấn luyện mô hình
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    verbose=1
)

# 7. Đánh giá trên tập test
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# 8. Vẽ đồ thị Loss và Accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Lấy class_indices từ train_generator
class_indices = train_generator.class_indices
print(f"Class indices: {class_indices}")

# 9. Lưu mô hình
model.save(r"C:\Users\ADMIN\PycharmProjects\pythonProject3\model_optimized2.h5")
